# Cleaning the data set

Now with a dataset store in the github project we can import it directly to our framework. We will continue using [pandas](https://pandas.pydata.org/)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/prope-2020-gh-classroom/practica-final-por-equipos-verano-2020-itam-EddOselotl/master/airbnb.csv")

In [3]:
df.head()

,id,name,host_id,host_since,host_total_listings_count,latitude,longitude,neighbourhood_cleansed,property_type,room_type,square_feet,price,review_scores_rating
0,22787,"Sunny suite w/ queen size bed, inside boutique...",87973,2010-03-03,9,19.44076,-99.16324,Cuauhtémoc,Boutique hotel,Private room,248.0,"$2,331.00",98.0
1,35797,Villa Dante,153786,2010-06-28,2,19.38399,-99.27335,Cuajimalpa de Morelos,Villa,Entire home/apt,32292.0,"$4,457.00",NaN
2,56074,Great space in historical San Rafael,265650,2010-10-19,2,19.43937,-99.15614,Cuauhtémoc,Condominium,Entire home/apt,646.0,$809.00,97.0
3,58955,Entire beautiful duplex in la Roma,282620,2010-11-09,1,19.42292,-99.15775,Cuauhtémoc,Loft,Entire home/apt,1184.0,"$1,932.00",100.0
4,61792,Spacious Clean Quiet room (own bath) in la Con...,299558,2010-11-26,1,19.41259,-99.17959,Cuauhtémoc,House,Private room,161.0,"$1,364.00",98.0


Now we need to check our dataframe types

In [4]:
df.dtypes

id                             int64
name                          object
host_id                        int64
host_since                    object
host_total_listings_count      int64
latitude                     float64
longitude                    float64
neighbourhood_cleansed        object
property_type                 object
room_type                     object
square_feet                  float64
price                         object
review_scores_rating         float64
dtype: object

As we can see, columns host_since and price are not the right data type, we want to change those only

In [5]:
### Change column datatype
df['host_since'] = pd.to_datetime(df['host_since'])

For the price column we need to remove the $\$$ sign and the commas before change its data type

In [6]:
### Remove $ sign
df['price'].replace({'\$': ''}, inplace=True, regex=True)
### Remove commas
df['price'].replace({',': ''}, inplace=True, regex=True)
### Change column datatype
df['price'] = df['price'].astype(float)

In [7]:
df.dtypes

id                                    int64
name                                 object
host_id                               int64
host_since                   datetime64[ns]
host_total_listings_count             int64
latitude                            float64
longitude                           float64
neighbourhood_cleansed               object
property_type                        object
room_type                            object
square_feet                         float64
price                               float64
review_scores_rating                float64
dtype: object

### Removing outliers

Removing outliers is a important part of the process of cleaning the data, it cab avoid us many troubles in the future when we try to calculate some statistical mesasures.

**Note: next definitions and implementations are based on [How to Remove Outliers in Python](https://kanoki.org/2020/04/23/how-to-remove-outliers-in-python/)**

*Outliers are the values in dataset which standouts from the rest of the data. The outliers can be a result of error in reading, fault in the system, manual error or misreading.*



Using the [describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) function we can easily see how our data set looks. The presence of some price values very high and some price values equals to zero may affect other measures like mean or standar deviation.

In [9]:
df.describe()

,id,host_id,host_total_listings_count,latitude,longitude,square_feet,price,review_scores_rating
count,2.166200e+04,2.166200e+04,21662.000000,21662.000000,21662.000000,71.00000,21662.000000,16479.000000
mean,2.811158e+07,1.207764e+08,6.394054,19.398241,-99.167641,1031.71831,1246.304404,95.063171
std,1.118437e+07,9.658871e+07,19.665067,0.045359,0.034666,3851.00801,4186.037111,8.680892
min,2.278700e+04,7.365000e+03,0.000000,19.168170,-99.340270,0.00000,0.000000,20.000000
25%,1.981078e+07,3.616692e+07,1.000000,19.373910,-99.179940,0.00000,407.000000,94.000000
50%,2.991470e+07,1.002619e+08,2.000000,19.410620,-99.168160,237.00000,713.000000,97.000000
75%,3.811844e+07,1.901580e+08,4.000000,19.429780,-99.153102,850.50000,1296.000000,100.000000
max,4.352751e+07,3.469379e+08,491.000000,19.557340,-98.968570,32292.00000,350007.000000,100.000000


First we need to remove those rows whose price value is less than one, as they can affect our further analisys.

In [10]:
df = df.loc[df['price'] > 0]

#### Removing outliers with $Z$ score

$Z$ score is an important measurement or score that tells how many Standard deviation above or below a number is from the mean of the dataset. Any positive $Z$ score means the no. of standard deviation above the mean and a negative score means no. of standard deviation below the mean $Z$ score is calculate by subtracting each value with the mean of data and dividing it by standard deviation.

$$z = \frac{x-\mu}{\sigma}$$

The Mu and Sigma above is population mean and Standard deviation and not of sample.

Calculating Z score using scipy.

In [11]:
df['z_score'] = stats.zscore(df['price'])

In [12]:
df.sort_values('price',ascending=False).head(10)

,id,name,host_id,host_since,host_total_listings_count,latitude,longitude,neighbourhood_cleansed,property_type,room_type,square_feet,price,review_scores_rating,z_score
7840,23895520,Habitación a 40min del centro historico,173103110,2018-02-12,1,19.33051,-98.97561,Iztapalapa,House,Private room,NaN,350007.0,NaN,83.303033
16748,38733276,Room in the most exclusive area in Mexico City,88593780,2016-08-07,1,19.44228,-99.20031,Miguel Hidalgo,Apartment,Private room,NaN,226302.0,100.0,53.755508
14355,35302200,En un lindo pueblito a 20 minutos de Xochimilco .,228126920,2018-11-28,1,19.25915,-99.12908,Xochimilco,Dome house,Private room,NaN,196149.0,NaN,46.553321
7859,23920872,Amazing Penthouse in Polanco.,55605035,2016-01-21,5,19.43697,-99.19851,Miguel Hidalgo,Apartment,Entire home/apt,NaN,181829.0,98.0,43.132922
9061,26454830,Hermoso Depto. en Santa Fe con Servicios Hotel...,198926402,2018-06-29,1,19.36001,-99.27918,Cuajimalpa de Morelos,Apartment,Entire home/apt,NaN,100002.0,60.0,23.588155
19528,41772248,Great apartment with POOL ★ MAID ★ GYM,61447972,2016-03-03,0,19.42638,-99.22442,Miguel Hidalgo,Condominium,Entire home/apt,NaN,100002.0,97.0,23.588155
1534,9578395,Best PH in the city!,38704347,2015-07-16,2,19.41523,-99.15852,Cuauhtémoc,Apartment,Entire home/apt,NaN,98381.0,NaN,23.200972
7626,23498353,"CASA EN PEDREGAL , COLONIAL MEXICANO",28935792,2015-03-07,4,19.32622,-99.20628,Álvaro Obregón,House,Entire home/apt,NaN,79743.0,NaN,18.749197
20171,42329864,Habitaciones estudiantes fuentes brotantes,336479023,2020-02-16,1,19.29043,-99.17503,Tlalpan,House,Private room,NaN,76519.0,NaN,17.979130
16950,38957187,The Best Penthouse in Mexico City. By Rick Joy,111850357,2017-01-16,1,19.43528,-99.19366,Miguel Hidalgo,Condominium,Entire home/apt,NaN,68186.0,100.0,15.988753


Now as per the empirical rule any absolute value of $Z$ score above 3 is considered as an Outlier. So we want to remove those rows whose $Z$ score is greater than 3

In [13]:
df = df.loc[df['z_score'].abs()<=3]

Now our data set describe looks more consisten

In [14]:
df.describe()

,id,host_id,host_total_listings_count,latitude,longitude,square_feet,price,review_scores_rating,z_score
count,2.157200e+04,2.157200e+04,21572.000000,21572.000000,21572.000000,70.000000,21572.000000,16439.000000,21572.000000
mean,2.810705e+07,1.207037e+08,6.381235,19.398220,-99.167631,1045.071429,1094.957723,95.061683,-0.036260
std,1.118104e+07,9.655023e+07,19.622541,0.045367,0.034630,3877.157574,1248.864035,8.682695,0.298297
min,2.278700e+04,7.365000e+03,0.000000,19.168170,-99.340270,0.000000,22.000000,20.000000,-0.292541
25%,1.981210e+07,3.616692e+07,1.000000,19.373868,-99.179892,0.000000,401.000000,94.000000,-0.202015
50%,2.990944e+07,1.002619e+08,2.000000,19.410610,-99.168140,237.000000,705.000000,97.000000,-0.129403
75%,3.810157e+07,1.900038e+08,4.000000,19.429780,-99.153090,855.750000,1293.000000,100.000000,0.011043
max,4.352733e+07,3.469379e+08,491.000000,19.557340,-98.968570,32292.000000,13637.000000,100.000000,2.959466


Finally we save the clean data set and is ready for further analisys.

In [16]:
df.to_csv("airbnb_clean.csv", index=False)